In [23]:
import pandas as pd
import plotly.express as px
import packaging


import dash
from dash import dcc
from dash import html

In [24]:
from dash.dependencies import Input, Output


In [25]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [26]:
grouped = spacex_df['Launch Site'].drop_duplicates()

grouped1 = spacex_df['Launch Site'].drop_duplicates()
print(grouped1)

0      CCAFS LC-40
26     VAFB SLC-4E
36      KSC LC-39A
49    CCAFS SLC-40
Name: Launch Site, dtype: object


In [27]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(
                                    id='site-dropdown',
                                    options = [
                                        {'label': 'All Sites', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                        ],
                                    value='ALL',
                                    placeholder="Select a Launch Site here",
                                    searchable=True
                                    ),           
                                                                
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(min = 0, max = 10000, step = 1000,
                                                id='payload-slider',
                                                value=[min_payload, max_payload]),
                                                                                
                                
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])


@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])

def get_scatter_chart(entered_site, entered_payload):
    low, high = entered_payload
    filtered_df= spacex_df[spacex_df['Payload Mass (kg)'] >= low] 
    filtered_df= filtered_df[filtered_df['Payload Mass (kg)'] <= high] 
    #filtered_df= spacex_df
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x ='Payload Mass (kg)',
                         y ='class',
                         color="Booster Version Category",
                         title='Correlation between payload and success rate for all sites')
        return fig
    else:
        data = filtered_df[spacex_df['Launch Site']==entered_site]
        fig = px.scatter(data, x ='Payload Mass (kg)',
                         y ='class',
                         color="Booster Version Category",
                         title='Correlation between payload and success rate for site {}'.format(entered_site))   
    return fig 


# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        data= filtered_df.groupby('Launch Site')['class'].sum().reset_index() 
        fig = px.pie(data, values='class',
                     names='Launch Site',
                     title='Total success launches by site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
        data = filtered_df.groupby('class')['Launch Site'].count().reset_index() 
        fig = px.pie(data, values='Launch Site',
                     names='class',
                     title='Total success launches for site {}'.format(entered_site))
    return fig           


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

    

   

KSC LC-39A has the largest successful launches
KSC LC-39A has the highest launch success rate
The payloads between 1900kg and 3700kg have the highest launch success rate
The payloads over 5300kg have the lowest launch success rate
The F9 Booster version : FT, has the highest launch success rate

In [28]:
data= spacex_df.groupby('Launch Site')['class'].sum().reset_index() 
data

,Launch Site,class
0,CCAFS LC-40,7
1,CCAFS SLC-40,3
2,KSC LC-39A,10
3,VAFB SLC-4E,4


In [29]:
 filtered_df = spacex_df[spacex_df['Launch Site']=='CCAFS LC-40']

filtered_df

data = filtered_df.groupby('class')['Launch Site'].count().reset_index()
data

,class,Launch Site
0,0,19
1,1,7
